In [1]:
import json

In [2]:
#open json file, try1.json is a input file
with open('try1.json', 'r') as file:
    data = json.load(file)
abstract_text1 = data.get('abstract', [])[0].get('text', '')
abstract_text2 = data.get('abstract', [])[1].get('text', '')
abstract_text3 = data.get('abstract', [])[2].get('text', '')
abstract_text4 = data.get('abstract', [])[3].get('text', '')
text = abstract_text1 + abstract_text2 + abstract_text3 + abstract_text4
text

'Background: Various transcription factors are involved in the process of mutually exclusive expression and clonal variation of the Plasmodium multigene (var) family. Recent studies revealed that a P. falciparum SWI/SNF-related matrix-associated actin-dependent regulator of chromatin (PfSWIB) might trigger stage-specific programmed cell death (PCD), and was not only crucial for the survival and development of parasite, but also had profound effects on the parasite by interacting with other unknown proteins. However, it remains unclear whether PfSIWB is involved in transcriptional regulation of this virulence gene and its functional properties.Methods: A conditional knockdown system "PfSWIB-FKBP-LID" was introduced to the parasite clone 3D7, and an integrated parasite line "PfSWIB-HA-FKBP-LID" was obtained by drug cycling and clone screening. Growth curve analysis (GCA) was performed to investigate the growth and development of different parasite lines during 96 h in vitro culturing, by

In [7]:
import nltk
import re
import ssl
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mumumumu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mumumumu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mumumumu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
#do data cleaning, and filter nouns in the text
text = re.sub(r'[^\w\s]', '', text)
text = re.sub(r'\d+', '', text)
text = text.lower()
text = re.sub(r'\b\w\b', '', text)
words_to_remove = ['background', 'method', 'result']
for word in words_to_remove:
    text = re.sub(r'\b' + re.escape(word) + r'\b', '', text, flags=re.IGNORECASE)

tokens = word_tokenize(text)
tagged_words = pos_tag(tokens)
nouns = [word for word, pos in tagged_words if pos.startswith('NN')]
stop_words = set(stopwords.words('english'))
words = nltk.word_tokenize(text)
text = ' '.join([word for word in words if word.lower() not in stop_words])

filtered_nouns = [noun for noun in nouns if noun.lower() not in stop_words]
filtered_nouns

['transcription',
 'factors',
 'process',
 'expression',
 'variation',
 'plasmodium',
 'multigene',
 'family',
 'studies',
 'falciparum',
 'regulator',
 'chromatin',
 'pfswib',
 'stagespecific',
 'cell',
 'death',
 'pcd',
 'survival',
 'development',
 'parasite',
 'effects',
 'parasite',
 'proteins',
 'pfsiwb',
 'regulation',
 'virulence',
 'gene',
 'propertiesmethods',
 'system',
 'pfswibfkbplid',
 'parasite',
 'clone',
 'parasite',
 'line',
 'pfswibhafkbplid',
 'drug',
 'cycling',
 'clone',
 'screening',
 'growth',
 'analysis',
 'gca',
 'growth',
 'development',
 'lines',
 'culturing',
 'parasitemia',
 'assays',
 'gene',
 'expression',
 'comparison',
 'groups',
 'expression',
 'pattern',
 'var',
 'genes',
 'lifecycle',
 'falciparum',
 'parasite',
 'lines',
 'addition',
 'rnaseq',
 'var',
 'gene',
 'expression',
 'linesresults',
 'gca',
 'knockdown',
 'pfswib',
 'growth',
 'development',
 'falciparum',
 'parasitemia',
 'pfswib',
 'decline',
 'culture',
 'pfswib',
 'lines',
 'qpcr',
 '

In [10]:
text

'various transcription factors involved process mutually exclusive expression clonal variation plasmodium multigene var family recent studies revealed falciparum swisnfrelated matrixassociated actindependent regulator chromatin pfswib might trigger stagespecific programmed cell death pcd crucial survival development parasite also profound effects parasite interacting unknown proteins however remains unclear whether pfsiwb involved transcriptional regulation virulence gene functional propertiesmethods conditional knockdown system pfswibfkbplid introduced parasite clone integrated parasite line pfswibhafkbplid obtained drug cycling clone screening growth curve analysis gca performed investigate growth development different parasite lines vitro culturing assessing parasitemia finally performed qpcr assays detect var gene expression profiling various comparison groups well mutually exclusive expression pattern var genes within single lifecycle falciparum different parasite lines addition r

In [11]:
import pandas as pd
df = pd.DataFrame({'abstract': [text],'nouns':[filtered_nouns]})
df['abstract'] = df['abstract'].apply(lambda x: nltk.word_tokenize(str(x).lower()))
df['nouns']

0    [transcription, factors, process, expression, ...
Name: nouns, dtype: object

In [12]:
from gensim import corpora, models
from pprint import pprint

In [13]:
#do LDA
data = df['nouns'].apply(lambda x: [str(x)] if not isinstance(x, list) else [str(item) for item in x])
dictionary = corpora.Dictionary(data)
corpus = [dictionary.doc2bow(doc) for doc in data]
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [14]:
#Select the most significant topics and made a list 
topics = lda_model.print_topics()
first_topic = lda_model.show_topic(0)
topic_list = [word for word, _ in first_topic]
topic_list

['parasite',
 'expression',
 'pfswib',
 'var',
 'genes',
 'gene',
 'falciparum',
 'lines',
 'development',
 'growth']

In [15]:
#write csv file
import csv

csv_file_path = 'word_list1.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(topic_list)
